In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
1.将survey_time数据提取出月份、日期、小时三个特征
2.将 family_income，income离散成5段的离散值
3.将 birth 转换成年龄值
4.调参：
    1.learning_rate=0.1，reg_lambda=10 0.5159674389480275 
"""

import xgboost as xgb
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_abbr.csv'
test_data_dile = 'data/happiness_test_abbr.csv'

In [3]:
df_train = pd.read_csv(data_dir+train_data_file)
df_test = pd.read_csv(data_dir+test_data_dile)
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,60000.0,2,2,1,2,3,3,2,4,3
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,40000.0,3,4,1,2,1,1,1,4,2
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,8000.0,3,3,1,2,3,2,1,4,2
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,12000.0,3,3,1,1,7,2,1,3,2
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,-2.0,4,3,1,1,1,3,2,3,-8


In [4]:
df_train = df_train.drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train[pd.to_datetime(df_train.survey_time).dt.month != 2]


In [5]:
df_train['month'] = pd.to_datetime(df_train.survey_time).dt.month
df_train['day'] = pd.to_datetime(df_train.survey_time).dt.day
df_train['hour'] = pd.to_datetime(df_train.survey_time).dt.hour

df_test['month'] = pd.to_datetime(df_test.survey_time).dt.month
df_test['day'] = pd.to_datetime(df_test.survey_time).dt.day
df_test['hour'] = pd.to_datetime(df_test.survey_time).dt.hour

df_train = df_train.drop('survey_time', 1)
df_test = df_test.drop('survey_time', 1)

df_train.head()

,happiness,survey_type,province,city,county,gender,birth,nationality,religion,religion_freq,...,house,car,marital,status_peer,status_3_before,view,inc_ability,month,day,hour
0,4,1,12,32,59,1,1959,1,1,1,...,1,2,3,3,2,4,3,8,4,14
1,4,2,18,52,85,1,1992,1,1,1,...,1,2,1,1,1,4,2,7,21,15
2,4,2,29,83,126,2,1967,1,0,3,...,1,2,3,2,1,4,2,7,21,13
3,5,2,10,28,51,2,1943,1,1,1,...,1,1,7,2,1,3,2,7,25,17
4,4,1,7,18,36,2,1994,1,1,1,...,1,1,1,3,2,3,-8,8,10,9


In [6]:
df_train.loc[1, :]

happiness              4.0
survey_type            2.0
province              18.0
city                  52.0
county                85.0
gender                 1.0
birth               1992.0
nationality            1.0
religion               1.0
religion_freq          1.0
edu                   12.0
income             20000.0
political              1.0
floor_area           110.0
height_cm            170.0
weight_jin           110.0
health                 5.0
health_problem         4.0
depression             3.0
hukou                  1.0
socialize              2.0
relax                  4.0
learn                  3.0
equity                 3.0
class                  6.0
work_exper             1.0
work_status            3.0
work_yr                2.0
work_type              1.0
work_manage            3.0
family_income      40000.0
family_m               3.0
family_status          4.0
house                  1.0
car                    2.0
marital                1.0
status_peer            1.0
s

In [7]:
sorted(df_train.family_income.unique())

[-3.0,
 -2.0,
 -1.0,
 0.0,
 300.0,
 400.0,
 950.0,
 1080.0,
 1100.0,
 1400.0,
 1500.0,
 2960.0,
 3150.0,
 3300.0,
 3400.0,
 3500.0,
 3516.0,
 4080.0,
 4420.0,
 4560.0,
 4700.0,
 4800.0,
 4860.0,
 5100.0,
 5600.0,
 5660.0,
 5820.0,
 6480.0,
 7800.0,
 8400.0,
 8800.0,
 8888.0,
 9500.0,
 9600.0,
 10500.0,
 13320.0,
 21600.0,
 23400.0,
 25200.0,
 25320.0,
 27600.0,
 27756.0,
 30200.0,
 32400.0,
 34000.0,
 34800.0,
 37000.0,
 38200.0,
 39300.0,
 40700.0,
 41300.0,
 42600.0,
 43560.0,
 43920.0,
 44000.0,
 44040.0,
 46400.0,
 48400.0,
 49440.0,
 50600.0,
 52800.0,
 55400.0,
 58200.0,
 62400.0,
 63000.0,
 64000.0,
 65040.0,
 77000.0,
 77016.0,
 79000.0,
 80400.0,
 83000.0,
 86000.0,
 88880.0,
 92400.0,
 93600.0,
 94800.0,
 102000.0,
 110400.0,
 112000.0,
 117600.0,
 134000.0,
 134400.0,
 138000.0,
 156000.0,
 181200.0,
 204000.0,
 220000.0,
 230000.0,
 320000.0,
 334000.0,
 360000.0,
 480000.0,
 750000.0,
 800000.0,
 960000.0,
 1320000.0,
 1500000.0,
 2000000.0,
 5000000.0,
 nan,
 450.0,
 500.

In [8]:
family_income_quantile = df_train.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_train['family_income'] = pd.cut(df_train.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])


In [9]:
income_quantile = df_train.income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_train['income'] = pd.cut(df_train.income, income_quantile, labels=[0, 1, 2, 3, 4])





In [10]:
family_income_quantile = df_test.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_test['family_income'] = pd.cut(df_test.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])
income_quantile = df_test.income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_test['income'] = pd.cut(df_test.income, income_quantile, labels=[0, 1, 2, 3, 4])



In [11]:
df_train['age']  = (2019 - df_train.birth)

In [12]:
df_train = df_train.drop('birth', 1)
df_train.loc[1, :]

happiness            4
survey_type          2
province            18
city                52
county              85
gender               1
nationality          1
religion             1
religion_freq        1
edu                 12
income               2
political            1
floor_area         110
height_cm          170
weight_jin         110
health               5
health_problem       4
depression           3
hukou                1
socialize            2
relax                4
learn                3
equity               3
class                6
work_exper           1
work_status          3
work_yr              2
work_type            1
work_manage          3
family_income        2
family_m             3
family_status        4
house                1
car                  2
marital              1
status_peer          1
status_3_before      1
view                 4
inc_ability          2
month                7
day                 21
hour                15
age                 27
Name: 1, dt

In [13]:
df_test['age']  = (2019 - df_test.birth)
df_test = df_test.drop('birth', 1)


In [14]:
len(df_train)
len(df_test)

2968

In [15]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]


In [16]:
len(df_train_validation)

1597

In [17]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()
tr_data

/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[3, 1, 1, ..., 1, 10, 23],
       [4, 1, 12, ..., 4, 15, 24],
       [5, 1, 21, ..., 19, 16, 39],
       ...,
       [3, 2, 29, ..., 23, 19, 65],
       [5, 1, 1, ..., 27, 9, 47],
       [4, 2, 18, ..., 12, 12, 30]], dtype=object)

In [18]:
tr_x_data = tr_data[:, 1:]
tr_y_data = tr_data[:, 0]

va_x_data = va_data[:, 1:]
va_y_data = va_data[:, 0]

In [19]:
len(tr_x_data[0])

42

In [20]:
mod = xgb.XGBRegressor(max_depth=6,
                       learning_rate=0.1,
                       n_estimators=100,
                       objective='reg:squarederror',
                       booster='gbtree',
                       gamma=5,
                       reg_lambda=10,
                       min_child_weight=1)

In [21]:
mod.fit(tr_x_data, tr_y_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=10, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [22]:
va_pred_data = np.round(mod.predict(va_x_data))
np.mean((va_pred_data - va_y_data)*(va_pred_data - va_y_data))

0.5159674389480275

In [23]:
pred_data = te_data[:, 1:]
pred_res = np.round(mod.predict(pred_data))

import csv
result = 'submission_908_xgb_tiaocan_1.csv'
with open(data_dir+result, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'happiness'])
    for i in range(len(te_data)):
        writer.writerow([int(te_data[i][0]), int(pred_res[i])])